In [1]:
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from sklearn.neighbors import DistanceMetric

In [2]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
data_raw_dir

'/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW/data/raw'

In [ ]:
os.listdir(data_raw_dir)

In [ ]:
df_remor = pd.read_csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=0)
df_remor.head()

In [ ]:
df_avg_drop_dist = df_remor[['LONGITUDE_ORIGINE', 'LATITUDE_ORIGINE', 'LONGITUDE_DESTINATION', 'LATITUDE_DESTINATION']]
df_avg_drop_dist
R = 6373.0

lat1 = np.radians(df_avg_drop_dist['LATITUDE_ORIGINE'])
lon1 = np.radians(df_avg_drop_dist['LONGITUDE_ORIGINE'])
lat2 = np.radians(df_avg_drop_dist['LATITUDE_DESTINATION'])
lon2 = np.radians(df_avg_drop_dist['LONGITUDE_DESTINATION'])

dlon = lon2 - lon1
dlat = lat2 - lat1
# print(dlon, dlat)
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
# print(type(a))
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
# print(c)
distance = R * c
# print(type(distance))
df_avg_drop_dist.loc[:, 'avg_dist'] = distance

# PYSPARK

In [3]:
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

In [4]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [5]:
spark = init_spark()
data = spark.read.csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=True)
data.head(5)
type(data)

pyspark.sql.dataframe.DataFrame

In [ ]:
data = data.withColumn('LONGITUDE_ORIGINE', F.expr('radians(LONGITUDE_ORIGINE)'))\
            .withColumn('LATITUDE_ORIGINE', F.expr('radians(LATITUDE_ORIGINE)'))\
                .withColumn('LONGITUDE_DESTINATION', F.expr('radians(LONGITUDE_DESTINATION)'))\
                    .withColumn('LATITUDE_DESTINATION', F.expr('radians(LATITUDE_DESTINATION)'))
data.head(5)

In [ ]:
# data = data.select('*', (data.LONGITUDE_DESTINATION - data.LONGITUDE_ORIGINE).alias('Diff_longitude'))
data = data.withColumn('Diff_long', F.expr('(LONGITUDE_DESTINATION-LONGITUDE_ORIGINE)/2'))\
            .withColumn('Diff_lat', F.expr('(LATITUDE_DESTINATION-LATITUDE_ORIGINE)/2'))
data.head(2)

In [ ]:
data = data.withColumn('LATITUDE_DESTINATION', F.expr('cos(LATITUDE_DESTINATION)'))\
            .withColumn('LATITUDE_ORIGINE', F.expr('cos(LATITUDE_ORIGINE)'))
data.head(2)

In [ ]:
data = data.withColumn('Diff_long', F.expr('sin(Diff_long)'))\
            .withColumn('Diff_lat', F.expr('sin(Diff_lat)'))
data.head(2)

In [ ]:
# a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
data = data.withColumn('A', F.expr('Diff_lat*Diff_lat + LATITUDE_DESTINATION * LATITUDE_ORIGINE * Diff_long * Diff_long'))
data.head(2)

In [ ]:
# c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
data = data.withColumn('One_minus_A', F.expr('1-A'))
data = data.withColumn('C', F.expr('2 * atan2( sqrt(A), sqrt(One_minus_A))'))
data = data.withColumn('Distance(Km)', F.expr('6373.0*C'))
data.head(2)

In [6]:
data = data.\
        withColumn('LONGITUDE_ORIGINE_rad', F.expr('radians(LONGITUDE_ORIGINE)')).\
        withColumn('LATITUDE_ORIGINE_rad', F.expr('radians(LATITUDE_ORIGINE)')).\
        withColumn('LONGITUDE_DESTINATION_rad', F.expr('radians(LONGITUDE_DESTINATION)')).\
        withColumn('LATITUDE_DESTINATION_rad', F.expr('radians(LATITUDE_DESTINATION)')).\
        withColumn('Diff_long', F.expr('(LONGITUDE_DESTINATION_rad-LONGITUDE_ORIGINE_rad)/2')).\
        withColumn('Diff_lat', F.expr('(LONGITUDE_DESTINATION_rad-LONGITUDE_ORIGINE_rad)/2')).\
        withColumn('LATITUDE_DESTINATION_cos', F.expr('cos(LATITUDE_DESTINATION_rad)')).\
        withColumn('LATITUDE_ORIGINE_cos', F.expr('cos(LATITUDE_ORIGINE_rad)')).\
        withColumn('Diff_long', F.expr('sin(Diff_long)')).\
        withColumn('Diff_lat', F.expr('sin(Diff_lat)')).\
        withColumn('A', F.expr('Diff_lat*Diff_lat + LATITUDE_DESTINATION_cos * LATITUDE_ORIGINE_cos * Diff_long * Diff_long')).\
        withColumn('One_minus_A', F.expr('1-A')).\
        withColumn('C', F.expr('2 * atan2( sqrt(A), sqrt(One_minus_A))')).\
        withColumn('Distance(Km)', F.expr('6373.0*C'))
data.head(10)

[Row(DATE_ORIGINE='2015-12-31T00:23:00', LONGITUDE_ORIGINE='-73.5872033592732', LATITUDE_ORIGINE='45.524980545696', RUE_ORIGINE='rue Drolet ', SECTEUR_ORIGINE=None, ARRONDISSEMENT_ORIGINE='Plateau-Mont-Royal', DATE_DESTINATION='2015-12-31T00:25:00', LONGITUDE_DESTINATION='-73.58679102366419', LATITUDE_DESTINATION='45.525130766625004', RUE_DESTINATION='rue de Grand-Pré ', SECTEUR_DESTINATION=None, ARRONDISSEMENT_DESTINATION='Plateau-Mont-Royal', MOTIF_REMORQUAGE="Constat d'infraction", LONGITUDE_ORIGINE_rad=-1.2843389859539491, LATITUDE_ORIGINE_rad=0.7945608024287601, LONGITUDE_DESTINATION_rad=-1.2843317893399486, LATITUDE_DESTINATION_rad=0.7945634242785765, Diff_long=3.5983070002443338e-06, Diff_lat=3.5983070002443338e-06, LATITUDE_DESTINATION_cos=0.700596354812899, LATITUDE_ORIGINE_cos=0.700598225651903, A=1.9303073431879138e-11, One_minus_A=0.9999999999806969, C=8.787052618939605e-06, Distance(Km)=0.055999886340502104),
 Row(DATE_ORIGINE='2015-12-31T00:24:00', LONGITUDE_ORIGINE='-73.

In [8]:
type(data), type(data.rdd)

(pyspark.sql.dataframe.DataFrame, pyspark.rdd.RDD)

In [12]:
os.environ

environ{'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
        'PYTHONIOENCODING': 'utf-8',
        'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python',
        'TERM_SESSION_ID': '2B23270F-0470-49FE-ADCF-42DA4CE8DC49',
        'CONDA_BACKUP_JAVA_HOME': '',
        '_CE_CONDA': '',
        'TERM_PROGRAM_VERSION': '433',
        'VSCODE_LOG_STACK': 'false',
        'LOGNAME': 'Gagandeep',
        'VSCODE_VERBOSE_LOGGING': 'true',
        'PATH': '/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/User/Gagandeep/.local/bin:/Library/TeX/texbin:/usr/local/go/bin:/usr/local/share/dotnet:~/.dotnet/tools:/Library/Frameworks/Mono.framework/Versions/Current/Commands:/Applications/Xamarin Workbooks.app/Contents/SharedSupport/path-bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/Library/Frameworks/Python.framework/Versions/3.7/bin',
        'VSCODE_IPC_HOOK_EXTHOST': '/var/folders/vp/677kztx911qcqv02qrbkfq0c0000gn/T/vscode-ipc-022bc81c-b17d-46f2-9d5e-a297